# Análise do dataset Pima Indians Diabetes Database

Colunas:

* Pregnancies: número de vezes grávida.
* Glucose: concentração de plasma glicose em duas horas em um teste oral de tolerância à glicose.
* BloodPressure: pressão diástolica (mm Hg).
* SkinThickness: espessura das dobras cutâneas do trícepes (mm).
* Insulin: duas horas serum insulina (mu U/ml)
* BMI: índice de massa corpórea (peso em kg/(altura em m)²)
* DiabetesPedigreeFunction: função de pedigree da diabetes.
* Age: idade (anos).
* Outcome: variável de classe (0 ou 1).

# Análise exploratória

In [ ]:
%matplotlib inline

# Importa biblioteca
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import warnings
from sklearn.model_selection import train_test_split
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

# Configuracoes basicas.
warnings.filterwarnings("ignore")
plt.rcParams["figure.dpi"] = 100
plt.style.use("ggplot")

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

# Configura seed.
np.random.seed(42)

## Leitura dos dados e primeira análise

In [ ]:
diabetes = pd.read_csv("../input/diabetes.csv")

In [ ]:
diabetes.head()

In [ ]:
diabetes.describe()

In [ ]:
#sns.pairplot(data=diabetes, hue="Outcome");

In [ ]:
#plt.figure(figsize=(12, 10))
#sns.heatmap(data=diabetes.corr(), annot=True)

In [ ]:
diabetes_features = diabetes.drop("Outcome", axis=1)
diabetes_outcome = diabetes["Outcome"]

In [ ]:
diabetes_features.head()

In [ ]:
diabetes_outcome.head()

# Criação de modelos

Agora vamos criar dois modelos simples que nos permitirão prever se uma pessoa pode ser considerada diabetica ou não.

O primeiro modelo é o [Naïve Bayes](http://scikit-learn.org/stable/modules/naive_bayes.html) e o segundo é o Support Vector Machine [SVM](http://scikit-learn.org/stable/modules/svm.html).

Antes, vamos separar os dados em treino e teste:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(diabetes_features, diabetes_outcome, test_size=0.2)

## Naïve Bayes

### Treinamento

In [ ]:
gnb = GaussianNB()

gnb.fit(X_train, y_train)

### Previsão

In [ ]:
y_predicted_gnb = gnb.predict(X_test)

### Avaliação do resultado

In [ ]:
confusion_matrix(y_test, y_predicted_gnb)
pd.crosstab(y_test, y_predicted_gnb, rownames=["Actual"], colnames=["Predicted"], margins=True)

In [ ]:
print(classification_report(y_test, y_predicted_gnb))

## SVM

### Treinamento

In [ ]:
svm = SVC()

param_grid = {'C': [1, 5, 10, 50],
              'gamma': [0.0001, 0.0005, 0.001, 0.005]}

grid = GridSearchCV(svm, param_grid)

grid.fit(X_train, y_train)

grid.best_params_

svm = grid.best_estimator_

### Previsão

In [ ]:
y_predicted_svm = svm.predict(X_test)

### Avaliação do resultado

In [ ]:
confusion_matrix(y_test, y_predicted_svm)
pd.crosstab(y_test, y_predicted_svm, rownames=["Actual"], colnames=["Predicted"], margins=True)

In [ ]:
print(classification_report(y_test, y_predicted_svm))

## O conjunto de teste

In [ ]:
X = X_test.assign(Outcome=y_test)

X

In [ ]:
case = 382

print(f"Outcome real: {y_test.loc[case]}")
print(f"Previsão do GaussianNB: {gnb.predict([X_test.loc[case, :]])[0]}")
print(f"Previsão do SVM: {svm.predict([X_test.loc[case, :]])[0]}")
print(f"Probabilidades pelo GaussianNB: {gnb.predict_proba([X_test.loc[case, :]])[0]}")